```
DATA_DIR/
    test/
        unknown/ - Need to move those
            img_<num>.jpg
    train/
        c0/
        ...
        c9/
            img_<num>.jpg
    [We need to create and move files by driver!]
    valid/
        c0/
        ...
        c9/
            img_<num>.jpg - valid_size
    [And then copy]
    sample/
        train/
            c0/
            ...
            c9/
                img_<num>.jpg - sample_train_size
        valid/
            c0/
            ...
            c9/
                img_<num>.jpg - sample_valid_size
        test/
            unknown/ - Need to move those
                img_<num>.jpg
```

In [4]:
# Prepare the statefarm data for training
# Assumes the data is already unzipped
DATA_DIR = '/home/ubuntu/courses/data/statefarm/'

In [5]:
# Create all the directories
for i in range(10):
    stri = str(i)
    %mkdir -p $DATA_DIR/valid/c$stri
    %mkdir -p $DATA_DIR/sample/valid/c$stri
    %mkdir -p $DATA_DIR/sample/train/c$stri
%mkdir -p $DATA_DIR/sample/test/unknown
%mkdir -p $DATA_DIR/test/unknown

In [6]:
# For keras we need to move all test files in a subdirectory unknown
%mv $DATA_DIR/test/ $DATA_DIR/unknown/
%mkdir $DATA_DIR/test
%mv $DATA_DIR/unknown/ $DATA_DIR/test/

/bin/sh: 1: mv: Argument list too long


In [7]:
# Now to create the validation set we better read the csv file
import pandas as pd
dil = pd.read_csv(DATA_DIR + 'driver_imgs_list.csv')

In [9]:
# From http://forums.fast.ai/t/statefarm-kaggle-comp/183/21
grouped = dil.groupby('subject')

In [22]:
len(grouped.groups)

26

In [18]:
import numpy as np
nb_drivers_valid = 3
subjects = [grouped.groups.keys()[i] for 
            i in np.random.randint(0, len(grouped.groups)-1, 
                                   nb_drivers_valid)]

In [19]:
subjects

['p015', 'p056', 'p064']

In [20]:
group = grouped.get_group(subjects[0])

In [24]:
group.values

array([['p015', 'c0', 'img_48693.jpg'],
       ['p015', 'c0', 'img_44903.jpg'],
       ['p015', 'c0', 'img_58514.jpg'],
       ..., 
       ['p015', 'c9', 'img_86098.jpg'],
       ['p015', 'c9', 'img_28068.jpg'],
       ['p015', 'c9', 'img_13997.jpg']], dtype=object)

In [26]:
import os
for sub in subjects:
    group = grouped.get_group(sub)
    for (s, cls, img) in group.values:
        source = DATA_DIR + 'train/%s/%s' % (cls, img)
        target = source.replace('train', 'valid')
        os.rename(source, target)

In [28]:
# Copy the sample files
sample_train_size = 100
sample_valid_size = 20
sample_test_size = 100
from shutil import copyfile
import glob
for i in range(10):
    fns = glob.glob(DATA_DIR + 'valid/c%i/*.jpg' % i)
    shuf = np.random.permutation(fns)
    for j in range(sample_valid_size): 
        copyfile(shuf[j], DATA_DIR + 'sample/valid/c%i/%s' % (i, shuf[j].split('/')[-1]))

In [29]:
for i in range(10):
    fns = glob.glob(DATA_DIR + 'train/c%i/*.jpg' % i)
    shuf = np.random.permutation(fns)
    for j in range(sample_train_size): copyfile(shuf[j], DATA_DIR + 'sample/train/c%i/%s' % (i, shuf[j].split('/')[-1]))

In [30]:
fns = glob.glob(DATA_DIR + 'test/unknown/*.jpg')
shuf = np.random.permutation(fns)
for j in range(sample_test_size): copyfile(shuf[j], DATA_DIR + 'sample/test/unknown/%s' % shuf[j].split('/')[-1])

In [ ]:
# All done! Nice!